# Script de pré-processamento para as bases de dados do ENEM

**Autor**: Rafael Victor Araujo Bernardes - rafaelvictor.bernardes@gmail.com

## Introdução

O seguinte _script_ tem como propósito realizar a preparação dos dados para a subsequente aplicação dos métodos de clusterização e seleção de atributos propostos em meu Trabalho de Conclusão de Curso (TCC). Para tanto, o _script_ realizará sequencialmente o processamento e as transformações dos dados. O processamento dos dados é uma etapa que tem como objetivo geral limpar, corrigir ou remover dados inconsistentes, verificar dados ausentes ou incompletos e identificar anomalias (_outliers_). Já o processo de transformação dos dados visa realizar a normalização, agregação, criação de novos atributos, redução e sintetização dos dados, entre outros. Estas etapas correspondem, respectivamente, à segunda e à terceira etapa do método _Knowledge Discovery in Databases_ (KDD) de Fayyad et al. (1996).

A princípio, o código aqui desenvolvido será aplicado nas bases de dados do ENEM referentes aos anos de 2022, 2020 e 2019. A escolha desses anos deve-se à alta correção dos atributos presentes nessas bases. Além disso, cada um desses anos está correlacionado a algum evento histórico cujo impacto na distribuição dos dados deseja-se investigar.

Espera-se que ao final da execução deste script, todas as bases alvo sejam transformada em bases menores, corrigidas, normalizadas e preparadas para aplicação de outras tecnicas.

## Importação dos dados e recursos necessários

As bases de dados utilizadas para o desenvolvimento deste trabalho (microdados) podem ser encontradas no portal do Instituto Nacional de Estudos e Pesquisas Educacionais Anísio Teixeira (INEP), o órgão responsável pelo ENEM, através do link: https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados/enem.

Cada microdado, por seu turno, contêm uma variedade de informações sobre os participantes coletadas ao longo de todo o processo do exame. Ao realizar o download, os usuários encontrarão não apenas a própria base de dados, mas também as provas, gabaritos, informações sobre questões, notas, questionários respondidos pelos inscritos, documentos técnicos e, acima de tudo, um extenso dicionário relacionado ao conjunto de dados. Este dicionário se mostra especialmente relevante para o desenvolvimento deste trabalho uma vez que ele caracteriza objetivamente todas as colunas presentes na base de dados.

Vale ressaltar que os microdados estão formatados em arquivos de extesão ".csv" e, para os anos selecionados, cada tabela contêm cerca de 76 colunas. Devido à grande quantidade de colunas e à diversidade de possíveis respostas que cada coluna pode conter, torna-se inviável realizar uma caracterização completa da base neste trabalho. No entanto, todas as colunas relevantes para o desenvolvimento desta pesquisa terão seus significados explorados em momentos oportunos. Recomenda-se a leitura dos dicionários de dados referêntes aos anos propostos.

In [ ]:
# !pip install category_encoders

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import category_encoders as ce
from category_encoders.one_hot import OneHotEncoder, OrdinalEncoder
from sklearn.cluster import KMeans

In [ ]:
# DATASET_ENEM_PATH = 'D:\Bases\MICRODADOS_ENEM_2019.csv'
# DATASET_ENEM_PATH = 'D:\Bases\MICRODADOS_ENEM_2020.csv'
DATASET_ENEM_PATH = 'D:\Bases\MICRODADOS_ENEM_2022.csv'

In [ ]:
microdadosEnem = pd.read_csv(DATASET_ENEM_PATH, sep=';', encoding='ISO-8859-1')

## Análise exploratória dos dados.

O objetivo principal desta análise exploratória dos dados é entender e extrair _insights_ iniciais sobre os microdados por meio da observação de padrões, tendências, relações e anomalias. Segue abaixo um compilado das etapas que segui:

In [ ]:
# Contato inicial com a base

microdadosEnem.head()

In [ ]:
# Visualizando as dimensões da base

microdadosEnem.shape

In [ ]:
# Visualizando as colunas da base

microdadosEnem.columns.values

## Pré-processamento dos dados

O objetivo principal deste pré-processamento dos dados é preparar e otimizar os dados brutos para análise ou modelagem, visando melhorar a qualidade, a eficácia e a eficiência das etapas subsequentes.

### Eliminando características individuais, de baixa variância ou inexpressivas.

Por meio do dicionário de dados, sabe-se que as colunas abaixo possuem pouca relevância para o objetivo da análise. Isto porque algumas delas possuem variância igual a zero (como é o caso da coluna "NU_ANO" que estará sempre preenchida com o ano de realização da prova) e outras representam caracteristicas altamente individuais dos candidatos e, portanto, não são aptas para avaliação de nenhum tipo de tendência (como é o caso das colunas com prefixo "TX_" que são os vetores com as respostas objetivas para as diferentes áreas de conhecimento da prova de cada candidato). Há ainda um conjunto de colunas inexpressivas que tratam sobre a cor de prova do participante. Todas elas serão removidas.

* "NU_INSCRICAO" - Número de inscrição (Individual),
* "NU_ANO" - Ano do Enem (Variância Zero),
* "TX_RESPOSTAS_CN" - Vetor com as respostas da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_RESPOSTAS_CH" - Vetor com as respostas da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_RESPOSTAS_LC" - Vetor com as respostas da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_RESPOSTAS_MT" - Vetor com as respostas da parte objetiva da prova de Matemática (Individual),
* "TX_GABARITO_CN" - Vetor com o gabarito da parte objetiva da prova de Ciências da Natureza (Individual),
* "TX_GABARITO_CH" - Vetor com o gabarito da parte objetiva da prova de Ciências Humanas (Individual),
* "TX_GABARITO_LC" - Vetor com o gabarito da parte objetiva da prova de Linguagens e Códigos (Individual),
* "TX_GABARITO_MT" - Vetor com o gabarito da parte objetiva da prova de Matemática (Individual),
* "CO_PROVA_CN" - Código do tipo de prova de Ciências da Natureza (Inexpressivo),
* "CO_PROVA_CH" - Código do tipo de prova de Ciências Humanas (Inexpressivo),
* "CO_PROVA_LC" - Código do tipo de prova de Linguagens e Códigos (Inexpressivo),
* "CO_PROVA_MT" - Código do tipo de prova de Matemática (Inexpressivo)


In [ ]:
microdadosEnem.drop(columns=[
    'NU_INSCRICAO',
    # 'NU_ANO', - Vai ser removido um pouco mais adiante
    'TX_RESPOSTAS_CN',
    'TX_RESPOSTAS_CH',
    'TX_RESPOSTAS_LC',
    'TX_RESPOSTAS_MT',
    'TX_GABARITO_CN',
    'TX_GABARITO_CH',
    'TX_GABARITO_LC',
    'TX_GABARITO_MT',
    'CO_PROVA_CN',
    'CO_PROVA_CH',
    'CO_PROVA_LC',
    'CO_PROVA_MT'
], inplace=True)

microdadosEnem.head()

### Eliminando redundâncias

Além disso, irei eliminar colunas redundantes.

Exemplo: "NO_MUNICIPIO_RESIDENCIA" (nome do município) equivalente a "CO_MUNICIPIO_RESIDENCIA" (código do município).

In [ ]:
microdadosEnem.drop(columns=[
    'NO_MUNICIPIO_ESC','CO_UF_ESC','SG_UF_ESC', # Todas estas colunas são iguais à CO_MUNICIPIO_ESC
    'NO_MUNICIPIO_PROVA','CO_UF_PROVA','SG_UF_PROVA', # Todas estas colunas são iguais à CO_MUNICIPIO_PROVA
    'TP_ANO_CONCLUIU' # Ano de Conclusão do Ensino Médio - Essa informação pode ser mais facilmente extraída da coluna TP_ST_CONCLUSAO
], inplace=True)

microdadosEnem.head()

### Tratamento de dados faltantes

In [ ]:
# Verificando se existem valores nulos

microdadosEnem.isnull().values.any()

True == A base contém dados nulos;
False == A base não contém dados nulos 

Para lidar com os dados ausentes no _dataset_, uma possível abordagem seria remover todas as linhas que contenham quaisquer valores nulos. Ao final deste processo, haveria um conjunto de dados compostos apenas por linhas que possuem todas as colunas preenchidas.

Apesar da simplicidade de implementação desta técnica, esta abordagem se mostrou inviável devido a alta perda de informações. Ao realizar um estudo de caso nas bases propostas, pude perceber que a implementação desta técnica ocarionaria na perda média de 70% dos registros presentes nas bases de dados.

Desse modo, será necessário adotar estratégias específicas para tratar cada coluna que contém valores nulos.
Primeiro é necessário compreender onde estão os valores nulos:

In [ ]:
# Verificando onde estão os valores nulos

microdadosEnem.isnull().sum()

##### ESTRATÉGIAS:

1. É possivel observar que há uma quantidade significativa de registros com informações ausentes relacionadas à **escola do candidato**. São elas:

    * "CO_MUNICIPIO_ESC" (Código do município da escola);
    * "TP_DEPENDENCIA_ADM_ESC" (Dependência administrativa da escola);
    * "TP_LOCALIZACAO_ESC" (Tipo de localização da escola); 
    * "TP_SIT_FUNC_ESC" (Situação de funcionamento da escola);
    * "TP_ENSINO" (Tipo de instituição de ensino);

Acredita-se que estas informações relacionadas a escola do candidato não eram de preenchimento obrigatório no momento cadastro do participante no exame. Isto justificaria a grande ausência de informações.

Dado que aproximadamente 70% das linhas da tabela total não possuem esses valores preenchidos, vou optar por remover essas colunas do modelo, pois não será possivel utiliza-las como objeto de ánalise confiável e nem fazer nenhum outro tipo de inferência preditiva ou classificativa.

Sendo assim, nenhum tipo de inferência ou analise sobre a escola do candidato será feita, mas conseguiremos preservar 70% a mais de dados.

_Uma sugestão de trabalho para o futuro é fazer analises relacionadas justamente as escolas dos participantes._

In [ ]:
# Removendo as colunas não confiáveis

microdadosEnem.drop(columns=[
    'CO_MUNICIPIO_ESC',
    'TP_DEPENDENCIA_ADM_ESC',
    'TP_LOCALIZACAO_ESC',
    'TP_SIT_FUNC_ESC',
    'TP_ENSINO'
], inplace=True)

2. As colunas relacionadas as notas dos participantes também demonstram ter uma quantidade grande de dados nulos. 

Acredita-se que estas colunas possuam registros vazios devido a ausência do candidato no dia de aplicação do exame. Desta forma, **vou optar por prosseguir apenas com os participantes presentes nos dois dias de aplicação da prova**.

Ao optar por isto, é fato que estarei reduzindo minha quantidade de registros, mas poderei também descartar as colunas relacionadas a presença dos candidatos e simplificar meu modelo.

(Estas colunas deverão ter variância zero. Ou seja, apenas candidatos presentes)

_Uma sugestão de trabalho para o futuro é fazer a analise justamente destas linhas que estou desconsiderando._

In [ ]:
# Removendo participantes ausêntes por meio da análise das notas

microdadosEnem.dropna(subset=['NU_NOTA_CN', 'NU_NOTA_CH', 'NU_NOTA_LC', 'NU_NOTA_MT', 'NU_NOTA_REDACAO'], inplace=True)

In [ ]:
# Validando se realmente posso remover as colunas de presença por meio da análise de variância.

colunasDePresença = ['TP_PRESENCA_CN', 'TP_PRESENCA_CH', 'TP_PRESENCA_LC', 'TP_PRESENCA_MT']
microdadosEnem[colunasDePresença].var()

In [ ]:
# Dado que a hipotese foi validada, posso remover as colunas de presença para simplificar meu modelo

microdadosEnem.drop(columns=colunasDePresença, inplace=True)

##### REAVALIAÇÃO DA CONSISTÊNCIA DOS DADOS:

Se tudo tiver ocorrido conforme o planejado, não devem ter mais dados nulos na tabela

In [ ]:
microdadosEnem.isnull().sum()

## Transformação dos dados

Esta pesquisa visa realizar uma análise histórica das bases de dados do ENEM e identificar tendências temporais nas características de candidatos quando dividios em _clusters_.

Em outras palavras, queremos entender se as caracteristicas relacionadas a _clusters_ de candidatos com predominância de notas baixas e altas se alterou ao longo do tempo.

Dessa forma, é necessário adotar uma forma simples para mensurar se a nota do partipante foi alta ou baixa. Isso será de grande importância para realização das analises quando os candidados estiverem clusterizados.

A estratégia adotada será a de criação de uma nova coluna que contenha a média simples de todas as notas que o candidato obteve.

In [ ]:
microdadosEnem['MEDIA_NOTAS'] = (
    microdadosEnem.NU_NOTA_CN +
    microdadosEnem.NU_NOTA_CH +
    microdadosEnem.NU_NOTA_LC +
    microdadosEnem.NU_NOTA_MT +
    microdadosEnem.NU_NOTA_REDACAO
)/5

In [ ]:
microdadosEnem.head()

Agora, seguirei com o descarte das colunas referênte as notas dos candidatos nas áreas do conhecimento, dado ques estas colunas já estão representadas na coluna de média.

Além disso, removerei as colunas relacionadas as competências da redação, dado que estas colunas nada mais são que componentes já representados pela coluna de média.

In [ ]:
microdadosEnem.drop(columns=[
    'NU_NOTA_CN',
    'NU_NOTA_CH',
    'NU_NOTA_LC',
    'NU_NOTA_MT',
    'NU_NOTA_REDACAO',
    'NU_NOTA_COMP1',
    'NU_NOTA_COMP2',
    'NU_NOTA_COMP3',
    'NU_NOTA_COMP4',
    'NU_NOTA_COMP5'
    
], inplace=True)

In [ ]:
microdadosEnem.head()

### Ajustes especificos para cada ano

#### Ajustes relacionados ao ano de 2022


1) Em 2022 a coluna "TP_COR_RACA" possuía uma opção a mais chamada "Não dispõe da informação" (Item: 6). 

Esta opção não existia nos anos anteriores. Acredito que seja possível normalizar os dados de 2022 para ficar mais coerente com os anos anteriores

**IDEA**: Unificar com a resposta "Não declarado" (Item: 0)

2) O ano de 2022 deixou te ter a resposta "Exterior" (item: 4) para a coluna "TP_ESCOLA"

**IDEA**: Para resolver esta questão, acredito que o melhor caminho seja remover de todas as demais tabelas os alunos que preencheram a opção "Exterior".

In [ ]:
primeiro_item = microdadosEnem.iloc[0]  # Obtendo o primeiro item do dataset

if primeiro_item['NU_ANO'] == 2022:
    microdadosEnem['TP_COR_RACA'].replace(6, 0, inplace=True)
    print("Itens 6 substituídos por 0 na coluna TP_COR_RACA.")
else:
    microdadosEnem = microdadosEnem[microdadosEnem['TP_ESCOLA'] != 4]
    print("Exterior removido como opção válida para a coluna - Tipo de escola do Ensino Médio")

microdadosEnem.head()
microdadosEnem.drop(columns=['NU_ANO'], inplace=True)

### Tratando variáveis categóricas

Necessário para que os algoritmos de clusterização e seleção de atributos funcionem corretamente.

In [ ]:
microdadosEnem.select_dtypes(include='object').describe()

#### Aplicação da técnica Get Dummies para tratar variáveis categóricas nominais

A função get_dummies do pandas é uma abordagem comum para codificar variáveis categóricas nominais em um formato numérico mais adequado para análises estatísticas e modelagem. Ao aplicar get_dummies a uma variável categórica, o pandas cria novas colunas binárias (0 ou 1) para cada categoria única presente na variável original.

Por exemplo, se você tiver uma coluna chamada "Cor" com categorias "Vermelho", "Verde" e "Azul", a função get_dummies criará três novas colunas: "Cor_Vermelho", "Cor_Verde" e "Cor_Azul". Cada linha terá um valor 1 na coluna correspondente à cor daquela linha e 0 nas outras colunas. Isso permite que as informações categóricas sejam tratadas como variáveis numéricas, facilitando a análise estatística e o uso em modelos de machine learning.

Em resumo, a função get_dummies transforma variáveis categóricas nominais em uma representação numérica que preserva as relações entre as categorias, tornando os dados mais adequados para análises quantitativas.

In [ ]:
microdadosEnem = pd.get_dummies(microdadosEnem, columns=['TP_SEXO'])

#### Aplicação da técnica Ordinal Encoding para tratar variáveis categóricas ordinais

O OrdinalEncoder, por sua vez, é uma técnica utilizada para codificar variáveis categóricas ordinais em valores numéricos. Este método atribui um valor numérico único a cada categoria, preservando a ordem natural das categorias.

Por exemplo, se você tiver uma coluna chamada "Educação" com categorias "Ensino Fundamental", "Ensino Médio" e "Graduação", o OrdinalEncoder atribuirá os valores 0, 1 e 2, respectivamente, para essas categorias.

A principal diferença entre o OrdinalEncoder e o get_dummies é que o primeiro é mais adequado para variáveis categóricas ordinais, onde a ordem das categorias possui um significado específico. Já o get_dummies é mais apropriado para variáveis categóricas nominais, onde não existe uma ordem intrínseca entre as categorias.

In [ ]:
ordinalEncoder = OrdinalEncoder()

colunas_ordinais = [
    'Q001', 'Q002', 'Q003', 'Q004', 'Q005', 'Q006',
    'Q007', 'Q008', 'Q009', 'Q010', 'Q011', 'Q012', 'Q013', 'Q014',
    'Q015', 'Q016', 'Q017', 'Q018', 'Q019', 'Q020', 'Q021', 'Q022',
    'Q023', 'Q024', 'Q025'
]

microdadosEnem[colunas_ordinais] = ordinalEncoder.fit_transform(microdadosEnem[colunas_ordinais])

Em resumo, o OrdinalEncoder codifica variáveis categóricas ordinais em valores numéricos, enquanto o get_dummies cria colunas binárias para cada categoria única em variáveis categóricas nominais. A escolha entre essas técnicas depende da natureza das variáveis e do contexto da análise.

In [ ]:
microdadosEnem.head()

In [ ]:
microdadosEnem.columns.values

In [ ]:
caminho_salvar = 'D:\\BASES_PRE_PROCESSADAS\\PRE_PROCESSADOS_ENEM_2022.csv'
microdadosEnem.to_csv(caminho_salvar, index=False)